# Exploration

In [13]:
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import polars as pl

In [14]:
def read_goodreads(path_goodreads_dir: Path) -> pl.DataFrame:
    """Read the Goodreads CSVs into a Polars DataFrame.

    The title and author columns are stripped of whitespace.

    Parameters
    ----------
    path_goodreads_dir : Path
        The directory containing the Goodreads CSVs.

    Returns
    -------
    pl.DataFrame
        The Goodreads data.
    """
    columns = {
        "Title": "title",
        "Author": "author",
        "My Rating": "rating",
        "Average Rating": "average_goodreads_rating",
        "Original Publication Year": "original_publication_year",
        "Number of Pages": "number_of_pages",
    }
    q = (
        pl.scan_csv(path_goodreads_dir, include_file_paths="path")
        .filter(pl.col("Exclusive Shelf") == "read")
        .select([*columns.keys(), "path"])
        .rename(columns)
        .with_columns(
            pl.col("title").str.strip_chars(),
            pl.col("author").str.strip_chars(),
        )
    )
    return q.collect()


def read_bookclub(path_bookclub: Path) -> pl.DataFrame:
    """Read the Bookclub CSV into a Polars DataFrame.

    The date column is converted to a date, and the title and author columns
    are stripped of whitespace.

    Parameters
    ----------
    path_bookclub : Path
        Path to the Bookclub CSV.

    Returns
    -------
    pl.DataFrame
        The Bookclub data.
    """
    columns = {
        "Nummer": "index",
        "Datum": "date",
        "Boek": "title",
        "Auteur": "author",
        "Wie heeft gekozen?": "blame",
        "Locatie": "location",
    }
    q = (
        pl.scan_csv(path_bookclub)
        .select(columns.keys())
        .rename(columns)
        .with_columns(
            pl.col("date").str.to_date("%m/%d/%Y"),
            pl.col("title").str.strip_chars(),
            pl.col("author").str.strip_chars(),
        )
    )
    return q.collect()


def pivot_goodreads_data(df_goodreads: pl.DataFrame) -> pl.DataFrame:
    """Pivot the Goodreads data, grouping by book, and calculating ratings.

    Parameters
    ----------
    df_goodreads : pl.DataFrame
        The Goodreads data.

    Returns
    -------
    pl.DataFrame
        The pivoted Goodreads data.
    """
    return (
        df_goodreads.pivot(
            "path",
            index=(
                "title",
                "author",
                "average_goodreads_rating",
                "original_publication_year",
                "number_of_pages",
            ),
            values="rating",
            aggregate_function="mean",
        )
        .rename(
            {
                "data/goodreads/koen_goodreads_library_export.csv": "Koen",
                "data/goodreads/thomas_goodreads_library_export.csv": "Thomas",
                "data/goodreads/koen_m_goodreads_library_export.csv": "Koen_M",
            }
        )
        .with_columns(
            pl.mean_horizontal("Koen", "Thomas", "Koen_M").alias("average_bookclub_rating")
        )
    )


def match_dataframes(
    df_bookclub: pl.DataFrame, df_pivot: pl.DataFrame, on: str, how: str
) -> pl.DataFrame:
    """Match the Bookclub and Goodreads DataFrames on a column.

    The match column is converted to lowercase before matching.
    After the match, the match column is dropped.

    Parameters
    ----------
    df_bookclub : pl.DataFrame
        The Bookclub DataFrame.
    df_pivot : pl.DataFrame
        The Goodreads DataFrame.
    on : str
        The column to match on.
    how : str
        The type of join to perform.

    Returns
    -------
    pl.DataFrame
        The matched DataFrame.
    """
    return (
        df_bookclub.with_columns(pl.col(on).str.to_lowercase().alias("temp_match_column"))
        .join(
            df_pivot.with_columns(pl.col(on).str.to_lowercase().alias("temp_match_column")),
            on="temp_match_column",
            how=how,
        )
        .drop("temp_match_column")
    )


def select_unmatched(df_bookclub: pl.DataFrame, df_scifi: pl.DataFrame) -> pl.DataFrame:
    """Select the rows in the Bookclub DataFrame that do not have a match in the Goodreads DataFrame.

    Parameters
    ----------
    df_bookclub : pl.DataFrame
        The Bookclub DataFrame.
    df_scifi : pl.DataFrame
        The matched DataFrame.

    Returns
    -------
    pl.DataFrame
        The unmatched rows.
    """
    return df_bookclub.filter(
        pl.col("title").is_in(
            df_scifi.filter(pl.col("average_goodreads_rating").is_null()).select("title")
        )
    )

In [15]:
# Paths to your CSV files
path_goodreads_dir = Path("data/goodreads")
# path_bookclub = Path("./data/boekenclub_overzicht.csv")
path_bookclub = Path("./data/boekenclub_overzicht_updated.csv")

# Read and combine Goodreads CSV files
df_goodreads = read_goodreads(path_goodreads_dir)
df_bookclub = read_bookclub(path_bookclub)

print(df_goodreads)
print(df_bookclub)

shape: (318, 7)
┌──────────────┬──────────────┬────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ title        ┆ author       ┆ rating ┆ average_good ┆ original_pub ┆ number_of_pa ┆ path         │
│ ---          ┆ ---          ┆ ---    ┆ reads_rating ┆ lication_yea ┆ ges          ┆ ---          │
│ str          ┆ str          ┆ i64    ┆ ---          ┆ r            ┆ ---          ┆ str          │
│              ┆              ┆        ┆ f64          ┆ ---          ┆ i64          ┆              │
│              ┆              ┆        ┆              ┆ i64          ┆              ┆              │
╞══════════════╪══════════════╪════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ Opvoedfeiten ┆ Emily Oster  ┆ 4      ┆ 4.14         ┆ 2019         ┆ 384          ┆ data/goodrea │
│ : Op data    ┆              ┆        ┆              ┆              ┆              ┆ ds/koen_good │
│ gebaseer…    ┆              ┆        ┆              ┆              ┆     

In [16]:
df_pivot = pivot_goodreads_data(df_goodreads)

print(df_pivot)

shape: (287, 9)
┌─────────────┬─────────────┬─────────────┬─────────────┬───┬──────┬────────┬────────┬─────────────┐
│ title       ┆ author      ┆ average_goo ┆ original_pu ┆ … ┆ Koen ┆ Koen_M ┆ Thomas ┆ average_boo │
│ ---         ┆ ---         ┆ dreads_rati ┆ blication_y ┆   ┆ ---  ┆ ---    ┆ ---    ┆ kclub_ratin │
│ str         ┆ str         ┆ ng          ┆ ear         ┆   ┆ f64  ┆ f64    ┆ f64    ┆ g           │
│             ┆             ┆ ---         ┆ ---         ┆   ┆      ┆        ┆        ┆ ---         │
│             ┆             ┆ f64         ┆ i64         ┆   ┆      ┆        ┆        ┆ f64         │
╞═════════════╪═════════════╪═════════════╪═════════════╪═══╪══════╪════════╪════════╪═════════════╡
│ Opvoedfeite ┆ Emily Oster ┆ 4.14        ┆ 2019        ┆ … ┆ 4.0  ┆ null   ┆ null   ┆ 4.0         │
│ n: Op data  ┆             ┆             ┆             ┆   ┆      ┆        ┆        ┆             │
│ gebaseer…   ┆             ┆             ┆             ┆   ┆      ┆       

In [17]:
df_scifi = match_dataframes(df_bookclub, df_pivot, "title", "inner")
df_unmatched = match_dataframes(df_bookclub, df_pivot, "title", "anti")

# export to csv
df_scifi.write_csv("data/clean/scifi.csv")
df_unmatched.write_csv("data/clean/unmatched.csv")

print(df_scifi)
print(df_unmatched)

shape: (59, 15)
┌───────┬────────────┬────────────────┬───────────────┬───┬──────┬────────┬────────┬───────────────┐
│ index ┆ date       ┆ title          ┆ author        ┆ … ┆ Koen ┆ Koen_M ┆ Thomas ┆ average_bookc │
│ ---   ┆ ---        ┆ ---            ┆ ---           ┆   ┆ ---  ┆ ---    ┆ ---    ┆ lub_rating    │
│ i64   ┆ date       ┆ str            ┆ str           ┆   ┆ f64  ┆ f64    ┆ f64    ┆ ---           │
│       ┆            ┆                ┆               ┆   ┆      ┆        ┆        ┆ f64           │
╞═══════╪════════════╪════════════════╪═══════════════╪═══╪══════╪════════╪════════╪═══════════════╡
│ 71    ┆ 2024-09-11 ┆ Galapagos      ┆ Kurt Vonnegut ┆ … ┆ 3.0  ┆ 4.0    ┆ null   ┆ 3.5           │
│ 28    ┆ 2018-06-06 ┆ 1Q84 Book 1    ┆ Haruki        ┆ … ┆ 4.0  ┆ null   ┆ null   ┆ 4.0           │
│       ┆            ┆ (1Q84, #1)     ┆ Murakami      ┆   ┆      ┆        ┆        ┆               │
│ 69    ┆ 2024-05-24 ┆ His Master's   ┆ Stanislaw Lem ┆ … ┆ 3.0  ┆ 3.0    ┆

In [18]:
df_plot = df_scifi.with_columns([pl.col("original_publication_year").cast(pl.Int32).alias("year")])
df_plot = df_plot.sort("year")
px.line(
    df_plot,
    x="year",
    y="average_bookclub_rating",
    title="Our Average Rating Over Time",
    markers=True,
)

In [23]:
selected_book = "Concept M"

df_plot_book = df_scifi.filter(pl.col("title") == selected_book)

categories = ["Koen", "Thomas", "Koen_M"]
values = [df_plot_book.select(member).to_series().item() for member in categories]

fig = go.Figure()
fig.add_trace(go.Scatterpolar(r=values, theta=categories, fill="toself", name=selected_book))
fig.update_layout(
    polar={"radialaxis": {"visible": True, "range": [0, 5]}},
    showlegend=False,
    title=f"Ratings for {selected_book}",
)
fig.show()

In [24]:
# Ensure the 'date' column is in datetime format
# df_plot = (
#     df_scifi.to_pandas()
# )  # If df_scifi is in Polars, convert it to Pandas, otherwise skip this step
# df_plot["date"] = pd.to_datetime(df_plot["date"])

# Sort the dataframe by date
df_plot = df_plot.sort_values("date")

# Create a Plotly figure
fig = go.Figure()

# Add a line trace for the rating over time, with hover information for the book title
fig.add_trace(
    go.Scatter(
        x=df_plot["date"],
        y=df_plot["average_bookclub_rating"],
        mode="lines+markers",
        name="Rating",
        text=df_plot["title"],  # Add the book title for hover information
        hovertemplate="<b>%{text}</b><br>Rating: %{y}<br>Date: %{x|%Y-%m-%d}<extra></extra>",
    )
)

# Add a line trace for the average Goodreads rating over time
fig.add_trace(
    go.Scatter(
        x=df_plot["date"],
        y=df_plot["average_goodreads_rating"],
        mode="lines+markers",
        name="Goodreads Rating",
        text=df_plot["title"],  # Add the book title for hover information
        hovertemplate="<b>%{text}</b><br>Goodreads Rating: %{y}<br>Date: %{x|%Y-%m-%d}<extra></extra>",
        line={"dash": "dash"},  # Optionally make the Goodreads rating line dashed
    )
)

# Set the title and labels for axes
fig.update_layout(
    title="Rating Over Time", xaxis_title="Year", yaxis_title="Rating", template="plotly_white"
)

# Show the interactive plot
fig.show()